In [1]:
import gc
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from concurrent.futures import ThreadPoolExecutor

2025-05-05 06:03:45.260694: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746425025.456326      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746425025.511080      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
test_df = pd.read_parquet('avito-for-dl-train.parquet')

In [ ]:
model1 = SentenceTransformer("deepvk/USER-bge-m3", device='cuda:0')
model2 = SentenceTransformer("deepvk/USER-bge-m3", device='cuda:1')

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
batch_size = 312
cossims = []

def encode_on_device(model, sentences, device):
    return model.encode(
        sentences,
        batch_size=batch_size,
        normalize_embeddings=True,
        convert_to_numpy=True,
        show_progress_bar=False,
        device=device
    )

for i in tqdm(range(0, len(test_df), batch_size)):
    batch = test_df.iloc[i:i+batch_size]
    
    sentences1 = batch['sentence1'].tolist()
    sentences2 = batch['sentence2'].tolist()
    
    with ThreadPoolExecutor(max_workers=2) as executor:
        future_embs1 = executor.submit(encode_on_device, model1, sentences1, torch.device('cuda:0'))
        future_embs2 = executor.submit(encode_on_device, model2, sentences2, torch.device('cuda:1'))
        
        embs1 = future_embs1.result()
        embs2 = future_embs2.result()
    
    batch_cossims = np.einsum('ij,ij->i', embs1, embs2)
    cossims.extend(batch_cossims.tolist())
    
    del embs1, embs2, batch
    gc.collect()

  0%|          | 0/1205 [00:00<?, ?it/s]

In [ ]:
test_df['userbge_cossim'] = cossims

In [ ]:
test_df[['variantid_1', 'variantid_2', 'userbge_cossim']].to_parquet('userbge_cossims_test.parquet')